## 关于梯度的调试
![](切线调试.png)
![](梯度调试公式推导.png)
上述公式计算梯度的效率很低，所以用作调试  
调试的出来的结果作为标准，用与和公式求出来的梯度做比较

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
np.random.seed(666)
X = np.random.random(size=(1000,10))
true_theta = np.arange(1,12,dtype=float)
X_b = np.hstack([np.ones([len(X),1]),X])
y = X_b.dot(true_theta) + np.random.normal(size=1000)

In [4]:
X.shape

(1000, 10)

In [5]:
y.shape

(1000,)

In [6]:
true_theta

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])

In [7]:
def J(theta, X_b, y):
    try:
        return np.sum((y - X_b.dot(theta)) ** 2) / len(X_b)
    except:
        return float('inf')

In [8]:
# 用数学推导所得的梯度
def dJ_math(theta, X_b, y):
    # 向量化实现
    return X_b.T.dot(X_b.dot(theta) - y) * 2 / len(X_b)

In [14]:
# 调试所用的梯度
def dJ_debug(theta, X_b, y,epsilon=0.01):
    res = np.empty(len(theta))
    for i in range(len(theta)):
        theta_1 = theta.copy()
        theta_1[i] += epsilon
        
        theta_2 = theta.copy()
        theta_2[i] -= epsilon
        
        res[i] = (J(theta_1,X_b,y) - J(theta_2,X_b,y)) / (2*epsilon)
    
    return res

In [15]:
def gradient_descent(dJ, X_b, y, initial_theta, eta, n_iter=1e4, epsilon=1e-8):
    theta = initial_theta
    i_iter = 0
    while i_iter < n_iter:
        gradient = dJ(theta, X_b, y)
        last_theta = theta
        theta = theta - eta * gradient
        if (abs(J(theta, X_b, y) - J(last_theta, X_b, y)) < epsilon):
            break
        i_iter += 1
    return theta

In [16]:
X_b = np.hstack([np.ones([len(X),1]), X])
initial_theta = np.zeros(X_b.shape[1])
eta = 0.01
%time theta_debug = gradient_descent(dJ_debug,X_b,y,initial_theta,eta)
theta_debug

Wall time: 5.83 s


array([ 1.1251597 ,  2.05312521,  2.91522497,  4.11895968,  5.05002117,
        5.90494046,  6.97383745,  8.00088367,  8.86213468,  9.98608331,
       10.90529198])

In [17]:
%time theta_math = gradient_descent(dJ_math,X_b,y,initial_theta,eta)
theta_math

Wall time: 782 ms


array([ 1.1251597 ,  2.05312521,  2.91522497,  4.11895968,  5.05002117,
        5.90494046,  6.97383745,  8.00088367,  8.86213468,  9.98608331,
       10.90529198])

# 重要
- dJ_math 只适合当前线性回归的损失函数
- 而dJ_debug适合所有函数